In [1]:
# aa1 stuff
from aa1 import DataLoader
from aa1 import extract_features
from aa1 import check_output

from aa2 import Trainer
from aa2 import parallel_coordinates
from aa2 import model

In [2]:
import torch

device = torch.device('cuda:1')

In [3]:
# assigment 1 stuff
dataset = DataLoader(data_dir="../../DDICorpus/DDICorpus/", device=device)
train_y, val_y, test_y = check_output(dataset.get_y()) 
train_X, val_X, test_X = check_output(extract_features(
                                                        data=dataset.data_df,
                                                        max_sample_length=dataset.max_sample_length,
                                                        id2word=dataset.id2word,
                                                        device=device
                                                        #Add any addtional arguments here
                                                       ))

In [4]:
#set up trainer
model_dump = "/tmp/aa2_models/" #you are allowed to change the dump_folder
trainer = Trainer(dump_folder=model_dump) 

In [5]:
# define a set of hyperparamaters
# test at least 5 different sets of hyperparamaters 
set_hyperparamaters = [{"learning_rate": 0.1,
                        "hidden_size": 1000,
                        "number_layers": 3,
                        "optimizer": "adam",
                        "batch_size": 50,
                        "bidirectional": 1
                       },
                       {"learning_rate": 0.02,
                        "hidden_size": 100,
                        "number_layers": 5,
                        "optimizer": "adam",
                        "batch_size": 50,
                        "bidirectional": 1
                       }
                        ]

In [6]:
model = model.LSTM_fixed_len(dataset.max_sample_length, 
                             set_hyperparamaters[0]['number_layers'], 
                             set_hyperparamaters[0]['hidden_size'], 
                             train_y.shape[1], 
                             set_hyperparamaters[0]['bidirectional'])

In [7]:
model

LSTM_fixed_len(
  (lstm): LSTM(103, 1000, num_layers=3, batch_first=True)
  (linear): Linear(in_features=1000, out_features=103, bias=True)
  (softmax): LogSoftmax()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [8]:
# hyperparamater tuning
# train you model with your set of hyperparamaters
for hp in set_hyperparamaters:
    trainer.train(train_X, train_y, val_X, val_y, model, hp)

RuntimeError: "fused_dropout" not implemented for 'Long'

In [ ]:
# create a parallel coordination plot over hyperparamaters
# add path to models and change metric to what ever metric you have chosen to use/want to use
parallel_coordinates(model_dump, metric="loss")

In [ ]:
# Based on the Parallel Coordination plot define 3 new hyperparamaters set thay you 
# think are worth testing
set_hyperparamaters_2 = [
                        # Example:
                        # {
                        #   "learning_rate": 0.1,
                        #   "number_layers": 3,
                        #   "optimizer": "adam"
                        # }
                        ]

In [ ]:
# train models for the new hyperparamaters
for hp in set_hyperparamaters_2:
    trainer.train(train_X, train_y, val_X, val_y, YOUR_MODEL_CLASS, hp)

In [ ]:
# create a parallel coordination plot over hyperparamaters again
parallel_coordinates(model_dump, metric="loss")

In [ ]:
#choose the best model base on the last parallel coordination plot
best_model_path = "PATH TO THE BEST MODEL"
scores = trainer.test(test_X, test_y, YOUR_MODEL_CLASS, best_model_path)